# Key Phrase Extraction #
This notebook explores techniques for effectively extracting keyphrases from texts. A few methods are explored, with code, sample output, commentary and some experiments along the way. If you'd like to see the final result, jump directly to the end of the notebook.


The novel 1984 (nineteen eighty-four) is used as a test text collection for this exploration.



## Pre-Processing ##
Load and process the text collection to get it ready for key phrase extraction.  

### Import Libraries ###

In [1]:
import nltk, re, pprint
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from collections import defaultdict
from string import punctuation
from difflib import SequenceMatcher

### Load Content ###

For testing only: I am using the novel 1984 (nineteen eighty-four) as my text collection to test out the different techniques.

In [2]:
# Load in the contents of text collection into a string in memory
with open('1984.rtf') as f:
    test_content = f.read()

# Pre-process to clean up the content
test_content = test_content.replace("\\par", "")
test_content = test_content.replace("&", "and")
test_content = test_content[171:]

For the desirable output, please replace "news.txt" with the target content source.

In [3]:
target_source = 'mystery_text_expository_2016.txt'
with open(target_source) as f:
    content = f.read()

### Tokenize ###  
Tokenize the text collection by sentences, which will help us do POS tagging later.

In [4]:
# Break down the content into a list of sentences
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sents = sent_tokenizer.tokenize(content)

In [5]:
# >>>>>>>>>>>>>>>>>>>> HELPER FUNCTIONS <<<<<<<<<<<<<<<<<<<<
# Tokenize each sentence
def word_tokenize2(sents):    
    pattern = r'''
      ([A-Z]\.)+            # abbreviations, e.g. U.S.A.
    | \w+'\w+               # contractions, e.g. I'm
    | \w+(-\w+)*            # words with optional internal hyphens
    | \$?\d+(\.\?d+)?%?     # currency and percentages, e.g. $12.40, 82%
    | \.\.\.                # ellipsis
    | [][.,;"'?():-_`]      # these are separate tokens; includes ], [
    '''
    
    matchIterators = [re.finditer(pattern, sent, re.VERBOSE) for sent in sents]
    
    # tokensBySent is a list of list where each list contains tokens in a sentence
    # tokens is a list of all tokens in the content
    tokensBySent = []
    tokens = []
    
    for matchIterator in matchIterators:
        words = [match.group(0) for match in matchIterator]
        tokensBySent.append(words)
        tokens += words
        
    return tokensBySent, tokens

In [6]:
tokensBySent, tokens = word_tokenize2(sents)

Create a words-only version of the tokenized collection for frequent terms analysis.

In [7]:
# remove all numbers and symbols
# wordsBySent is a list of list where each list contains words in a sentence
# words is a list of all words in the content
wordsBySent = [[token for token in tokens if re.match("[a-zA-Z]+", token)] for tokens in tokensBySent]
words = [token for token in tokens if re.match("[a-zA-Z]+", token)]

### POS (Part of Speech) Tagging ###
Use the nltk default tagger to tag the sentences.

In [8]:
# use a pre-trained tagger to tag the text collection
tagged_tokens = [nltk.tag.pos_tag(token) for token in tokensBySent]
tagged_sents = [nltk.tag.pos_tag(sent) for sent in wordsBySent]

***

## Interesting Words ... By Frequency? ##
#### a.k.a Showing Frequent Terms ####

What does an interesting word look like in a text collection? It probably occurs in the text just the right amount of times - less frequently than a stopword, but more often than just once or twice (so it is unlikely to be a typo or tangential thought) 

Words that are too frequent and/or short are filtered out, and then the most common 20 words are shown.  
Filtering criteria:  
1. Filter out all numbers and punctuations
2. Filter out all stop words
3. Filter out all words of length <= 4

In [9]:
# interestingWords is a list of normalized words where stopwords and very short words are removed
# removing stopwords (both lower and upper-case)
cachedStopWords = stopwords.words("english")
interestingWords = [word for word in words if word.lower() not in cachedStopWords]

# lowercasing and only showing words of a greater length
interestingWords = [word for word in interestingWords if len(word) >= 5]

In [10]:
# let's see some interesting words!
freqdist = nltk.FreqDist(interestingWords)
freqdist.most_common(20)

[('Congress', 2382),
 ('people', 2240),
 ('Government', 2095),
 ('States', 1999),
 ('world', 1711),
 ('United', 1701),
 ('American', 1639),
 ('would', 1614),
 ('years', 1481),
 ('great', 1434),
 ('country', 1399),
 ('peace', 1106),
 ('every', 1099),
 ('public', 1074),
 ('America', 1025),
 ('Federal', 998),
 ('national', 981),
 ('government', 938),
 ('power', 902),
 ('shall', 898)]

We start seeing a little bit of content there, but nothing particularly interesting... We can maybe increase the number of words we show. However, without context it is difficult to infer what the text is about, as most of these words can take on multiple meanings.

***

## Nouns and Beyond ##
#### a.k.a Information obtained from Syntax or Partial Parsing (Chunking) ####

### Proper Nouns ###

What about nouns? If we can see what objects are in the text, maybe it will give us a better idea of what the text is about?  

I explored that idea by extracting the most frequent proper nouns using chunking and displaying them here.

In [11]:
# >>>>>>>>>>>>>>>>>>>> HELPER FUNCTIONS <<<<<<<<<<<<<<<<<<<<
# Get the chunked text list using the given chunker specified by the grammar string and the label
# the grammar string and the label should contain the same keyword, e.g. 'PROPER_NOUN'
def get_chunked_text(grammar, tagged_sents, labels):
    # chunk parser (cp) for proper nouns
    cp = nltk.RegexpParser(grammar)
    chunked_text = []
    for sent in tagged_sents:
        if len(sent) > 0:
            for subtree in cp.parse(sent).subtrees():
                if subtree.label() in labels:
                    chunked_text += [text for (text, _) in subtree.leaves()]
    return chunked_text

In [12]:
# chunk parser (cp) for proper nouns
grammar_NP = r""" PROPER_NOUN:
    {<NNP.*>+}           
    """
proper_nouns = get_chunked_text(grammar_NP, tagged_sents, 'PROPER_NOUN')
nltk.FreqDist(proper_nouns).most_common(20)

[('Congress', 2382),
 ('Government', 1966),
 ('States', 1792),
 ('United', 1701),
 ('America', 1025),
 ('Federal', 879),
 ('Americans', 552),
 ('State', 548),
 ('President', 496),
 ('Department', 489),
 ('Union', 439),
 ('National', 355),
 ('Secretary', 319),
 ('Commission', 288),
 ('Navy', 285),
 ('Europe', 275),
 ('War', 272),
 ('Army', 256),
 ('Act', 254),
 ('American', 245)]

We start to see more interesting words, e.g. "Newspeak", "Police", but there's still words like "I'm", "Inner" that are ambiguous.

### Name Entities ###

How can we do better? What if we take a look into the special nouns (name entity words)? They might give us a better idea of the topics of the text.  

To get a meaningful list of name entities, I first parsed the text collection using the default nltk ne chunker to get a preliminary list. I then tried to deduplicate the list by combining similar words. The 20 most common name entities are then shown.

** Point of Reflection **  
Before the deduplication process, I decided to filter out name entity words occuring only once from the preliminary list. The goal for this step is to take out words that are incorrectly tagged and/or peripheral, thereby reducing noise in the deduplication step. Some key terms only occur once in a text collection, and this step unfortunately will remove them from the result. By trying it both ways, I feel that the filtering does more good than harm. I therefore choose to keep it and sacrifice recall for a better precision.

In [13]:
# >>>>>>>>>>>>>>>>>>>> HELPER FUNCTIONS <<<<<<<<<<<<<<<<<<<<
# Get a list of name entities with their occurence frequency from a list of tagged sentences
# input: a list of POS tagged sentences
# output: frequency distribution of (name entity, their label)
def get_name_entities(tagged_sents):
    # chunk collection using the default nltk ne chunker
    chunked_sents = [nltk.ne_chunk(sent) for sent in tagged_sents]
    
    name_entities = []
    
    # extract name entities from each sentence
    for sent in chunked_sents:
        for subtree in sent.subtrees():
            
            # nltk ne chunker returns three types of entities: GPE, ORGANIZATION, and PERSON
            # we want to retrieve words in any of the three categories
            if subtree.label() in ['GPE', 'ORGANIZATION', 'PERSON']:
                
                # if the subtree contains more than one token, combine the tokens together into one
                name_entity = ' '.join([word for (word, _) in subtree.leaves()])
                name_entities += [(name_entity, subtree.label())]
    
    # create a frequency distribution of the name entities
    return nltk.FreqDist(name_entities)

# Compare two entities and  
# 1. if they are similar, return the entity that best represents both entities
# 2. if they are dissimilar, return None
# entities a and b have the format ((word, name entity type), frequency)
def get_common_terms(a, b):
    a_term_list = a[0][0].split()
    b_term_list = b[0][0].split()
    
    term = None
    sum_freq = a[1] + b[1]
    
    if len(a_term_list) > 1 and len(b_term_list) > 1:
        similarity_score = SequenceMatcher(None, a[0][0], b[0][0]).ratio()     
        if similarity_score >= 0.75: term = a[0] if a[1] > b[1] else b[0]
    elif len(a_term_list) > 1 and b_term_list[0] in a_term_list: term = a[0]
    elif len(b_term_list) > 1 and a_term_list[0] in b_term_list: term = b[0]
    else:
        similarity_score = SequenceMatcher(None, a[0][0], b[0][0]).ratio()
        if similarity_score >= 0.9:
            term = a[0] if a[1] > b[1] else b[0]
    
    if term is None: return None
    else: return (term, sum_freq)

# Take a name entity list, and consolidate similar terms into one term with combined frequency, e.g.
# if the word 'Richard' appeared 12 times and 'Richard Thomas' appeared 20 times in a text,
# the function removes 'Richard' from the list, and only keeps 'Richard Thomas' with an updated frequency of 32
def dedup_ne_list(ne_list):
    new_ne_list = []
    while len(ne_list) > 0:
        term1 = ne_list.pop()
        last_common_term = term1
        i = 0
        while i < len(ne_list):
            term2 = ne_list[i]
            common_term = get_common_terms(last_common_term, term2)
            if common_term is not None:
                ne_list.remove(term2)
                last_common_term = common_term
            else: 
                i += 1
        new_ne_list += [last_common_term]
    
    new_ne_list = sorted([term for term in new_ne_list if term[1] > 1], key = lambda x: x[1], reverse = True)
    return new_ne_list

In [14]:
# get preliminary name entity list
freqdist = get_name_entities(tagged_tokens)

In [15]:
# filter out terms that only occur once to reduce noise
ne_freq_list = [(term, freqdist[term]) for term in freqdist if freqdist[term] > 1]

# deduplicate the list and return the top 20 words
new_ne_list = dedup_ne_list(ne_freq_list.copy())
new_ne_list[:20]

[(('National Congress', 'ORGANIZATION'), 2496),
 (('American Army', 'ORGANIZATION'), 1731),
 (('United States', 'GPE'), 1520),
 (('Latin America', 'PERSON'), 925),
 (('Federal Housing Administration', 'ORGANIZATION'), 532),
 (('State Department', 'ORGANIZATION'), 526),
 (('Applause', 'ORGANIZATION'), 385),
 (('Eastern Europe', 'GPE'), 249),
 (('Senate', 'ORGANIZATION'), 235),
 (('Nation', 'ORGANIZATION'), 233),
 (('Mainland China', 'GPE'), 223),
 (('Treasury Department', 'ORGANIZATION'), 212),
 (('Americans', 'GPE'), 209),
 (('Republic of Korea', 'ORGANIZATION'), 208),
 (('Navy', 'ORGANIZATION'), 204),
 (('George Washington', 'PERSON'), 177),
 (('Arab States', 'GPE'), 171),
 (('Cuba', 'GPE'), 171),
 (('House', 'ORGANIZATION'), 166),
 (('United Nations', 'ORGANIZATION'), 151)]

In [16]:
# >>>>>>>>>>>>>>>>>>>> OUTPUT <<<<<<<<<<<<<<<<<<<<
# create output string for name entity result
ne_output = [(ne[0][1], ne[0][0]) for ne in new_ne_list[:20]]
ne_output_string = ''
for ne in ne_output:
    ne_output_string += ne[1] + ', '
ne_output_string = ne_output_string[:-2]+ '\n\n'

We have now seen some key 'players' in the text. What are they doing? How are they related to each other? We explore that by looking into other aspects of the text. One way to get an overall understanding of the context of the text is to look into the concept space of the words in text, and see where they overlap.

Let's take a look at that.

***

## High Level Concepts ##
#### a.k.a Using Semantic Similarity / Finding Higher-Level Concepts ####

I used WordNet to navigate through the high level concepts of the text collection.  

1. For each word of interest in the text collection, its hypernyms are extracted and added into an overall list along with hypernyms of other words.  
2. For each hypernym in the overall list, I calculated their frequency and specificity.
3. The hypernyms are then assigned a score = - (1/frequency\* + 1/specificity\*), and the hypernyms with the 20 highest scores are shown.

*Notes*  
A substantial portion of the code is adopted from code by Anna Swigart, ANLP 2015  
\* adjusted by +1 to avoid 1/0 error

** Point of Reflection **  
So we have a list of hypernyms, how do we know which ones are the most useful in telling us the context of the text? I think there are two intuitive and easy to measure metrics - the frequency and specificity of the hypernym. If more words share the same hypernym, it is likely to be more representative of the context of the text. Additionally, if they hypernym is very specific, then it gives us more detailed information about the text. The next question is then how to rank the hypernyms based on their frequency and specificity. My first approach was to put hard limit on the values of the two metrics, i.e. throwing out a hypernym if its frequency and/or specificity are lower than a threshold. Finding that threshold, however, was a challenge since it is almost an arbitrary decision, and really depends on the specific text collection. I then decided to use a relative score (~ sum of inverse of frequency and specificity), calculate that for each hypernym, rank them by the score, and output the ones with the highest score. The second approach avoids the need for an arbitrary external threshold on the metrics, and relies on the relative usefulness of each hypernym against each other.

In [17]:
# >>>>>>>>>>>>>>>>>>>> HELPER FUNCTIONS <<<<<<<<<<<<<<<<<<<<
# adopted from code by Anna Swigart, ANLP 2015

# get n most frequent normalized unigrams of word_type (N for noun and V for verb) in tagged sentences
# if n is None, return full set of normalized unigrams of word_type
# word processing done for normalization: lemmatization, removing stop words, punctuation, removing short words
def get_unigrams(tagged_sents, word_type, n):
    wnl = WordNetLemmatizer()
    normed_tagged_words = [wnl.lemmatize(word[0].lower()) for sent in tagged_sents
                           for word in sent 
                           if word[0].lower() not in nltk.corpus.stopwords.words('english')
                           and word[0] not in punctuation 
                           and not re.search(r'''^[\.,;"'?!():\-_`]+$''', word[0])
                           and len(word[0])>= 5
                           and word[1].startswith(word_type)]
    if n is None: 
        normed_unigrams = set(normed_tagged_words)
    else:
        normed_unigrams = [word for (word, count) in nltk.FreqDist(normed_tagged_words).most_common(n)]
    return normed_unigrams

# get frequency distribution of hypernyms of tokens of given word_type in tagged sentences
# first the top normalized unigrams are extraced from the tagged sentences
# then for each unigram, add its list of hypernyms and add it to an overall hypernym list
# count the occurences of each hypernym and return a frequency distribution
def categories_from_hypernyms(sents, word_type, n = None):
    
    # We can specify a fixed length termlist by only using the n most frequent unigrams
    # I tested both and found very minimal difference between the two, so I decided to
    # use the full set
    termlist = get_unigrams(sents, word_type, n)
    
    hypterms_dict = defaultdict(list)
    for term in termlist:
        s = wn.synsets(term.lower(), word_type.lower())
        for syn in s:
            for hyp in syn.hypernyms():
                hypterms_dict[hyp.name].append(term)
    return hypterms_dict

# get the hypernyms and their frequencies from a given hypernym dictionary, 
# and for each hypernym, add information about its specificity, as defined by the 
# depth (min_depth) of its corresponding synset
# return the information as a list of triplets (hypernym, depth, frequencies / occurences)
def get_hypernym_info(hypernyms_dict):
    new_hypernyms_dict = []
    for hypernym in hypernyms_dict.keys():
        new_hypernyms_dict += [(hypernym(), 
                                wn.synset(hypernym()).min_depth(), 
                                len(hypernyms_dict.get(hypernym)))]
    
    # sort the list by specificity, i.e. the min depth of the specific hypernym
    new_hypernyms_dict = sorted(new_hypernyms_dict, key = lambda x: x[1], reverse = True)
    return new_hypernyms_dict

# returns a string with the names of hypernyms in a hypernym list
def print_hypernym(hypernyms_list):
    hypernyms = ''
    pattern = r"([a-zA-Z]+?)\."
    for triplet in hypernyms_list:
        match = re.search(pattern, wn.synset(triplet[0]).name())
        hypernyms += match.group(1) + ', '
    return hypernyms[:-2]+ '\n'

### Verbs ### 
Let's take a look at the action words!

In [18]:
# generate hypernym dictionary for the given tagged text 
verb_hypernyms_dict = categories_from_hypernyms(tagged_tokens, 'V')

# create a list of (hypernym, specificity, frequency) for hypernyms in the dictionary
verb_hypernyms = get_hypernym_info(verb_hypernyms_dict)

In [19]:
# rank hypernyms by - [1/(freq + 1) + 1/(specificity + 1)]
verb_hypernyms_filtered = sorted(verb_hypernyms, key = lambda x: - (1/(x[1]+1) + 1/(x[2]+1)), reverse = True)

# output the top 20 hypernyms
verb_hypernyms_filtered[:20]

[('challenge.v.02', 8, 28),
 ('call.v.05', 8, 24),
 ('express.v.01', 7, 35),
 ('command.v.02', 8, 18),
 ('mean.v.01', 7, 23),
 ('forbid.v.01', 9, 14),
 ('order.v.01', 7, 20),
 ('convey.v.01', 6, 30),
 ('protest.v.02', 6, 24),
 ('rede.v.02', 6, 22),
 ('request.v.02', 6, 22),
 ('provoke.v.03', 9, 10),
 ('hash_out.v.01', 5, 38),
 ('restrict.v.03', 5, 38),
 ('urge.v.01', 7, 13),
 ('publicize.v.01', 5, 32),
 ('broach.v.01', 6, 17),
 ('guarantee.v.04', 7, 12),
 ('imply.v.02', 8, 10),
 ('elaborate.v.01', 6, 15)]

In [20]:
# >>>>>>>>>>>>>>>>>>>> OUTPUT <<<<<<<<<<<<<<<<<<<<
# create output string for verb high level concept result
verb_hypernyms_output = verb_hypernyms_filtered[:20]
verb_hypernyms_output_string = print_hypernym(verb_hypernyms_output)
verb_hypernyms_output_synset = [wn.synset(triplet[0]) for triplet in verb_hypernyms_filtered[:20]]

### Nouns ### 
How about some nouns?

In [21]:
# generate hypernym dictionary for the given tagged text 
noun_hypernyms_dict = categories_from_hypernyms(tagged_tokens, 'N')

# create a list of (hypernym, specificity, frequency) for hypernyms in the dictionary
noun_hypernyms = get_hypernym_info(noun_hypernyms_dict)

In [22]:
# rank hypernyms by - [1/(freq + 1) + 1/(specificity + 1)]
noun_hypernyms_filtered = sorted(noun_hypernyms, key = lambda x: - (1/(x[1]+1) + 1/(x[2]+1)), reverse = True)

# output the top 20 hypernyms
noun_hypernyms_filtered[:20]

[('common_fraction.n.01', 9, 22),
 ('termination.n.05', 8, 29),
 ('beginning.n.05', 8, 22),
 ('improvement.n.02', 8, 22),
 ('motion.n.06', 7, 29),
 ('decrease.n.04', 8, 20),
 ('executive_department.n.01', 11, 12),
 ('increase.n.05', 8, 18),
 ('room.n.01', 7, 23),
 ('motion.n.03', 7, 22),
 ('production.n.07', 9, 13),
 ('support.n.10', 7, 20),
 ('position.n.06', 7, 20),
 ('device.n.01', 6, 32),
 ('large_integer.n.01', 6, 31),
 ('change.n.03', 6, 30),
 ('tract.n.01', 6, 29),
 ('blow.n.01', 9, 12),
 ('activity.n.01', 5, 91),
 ('payment.n.01', 7, 18)]

In [23]:
# >>>>>>>>>>>>>>>>>>>> OUTPUT <<<<<<<<<<<<<<<<<<<<
# create output string for noun high level concept result
noun_hypernyms_output = noun_hypernyms_filtered[:20]
noun_hypernyms_output_string = print_hypernym(noun_hypernyms_output)

It seems that the high level concept method is working better on verbs than nouns, at least for the test text collection. Maybe nouns are more abmiguous by nature, which make their hypernyms tell a less coherent story?  

So far our exploration has mostly revolved around information extraction from single words (except for chunking). While it gives us useful information, it is worth looking into words in relation to their neighbors.

***

## Collocations and Friends ##
#### a.k.a Collocations ####

Let's start with bigrams and trigrams. I used the default nltk collocation bigram and trigram finder, and filtered out punctuations, stop words and infrequent terms. 

### Bigrams and Trigrams ###

In [24]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# >>>>>>>>>>>>>>>>>>>> HELPER FUNCTIONS <<<<<<<<<<<<<<<<<<<<
# get the n top bigrams using nltk default collocation bigram finder
# punctuations, stop words and infrequent terms are filtered out
# pmi scoring measure is used
def bigram_collocations(words, n):
    finder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
    finder.apply_word_filter(lambda w: w[0] in punctuation)
    finder.apply_word_filter(lambda w: w.lower() in cachedStopWords)
    finder.apply_freq_filter(2)
    return finder.nbest(bigram_measures.pmi, n)

# get the n top trigrams using nltk default collocation bigram finder
# punctuations, stop words and infrequent terms are filtered out
# pmi scoring measure is used
def trigram_collocations(words, n):
    finder = nltk.collocations.TrigramCollocationFinder.from_words(words)
    finder.apply_word_filter(lambda w: w[0] in punctuation)
    finder.apply_word_filter(lambda w: w.lower() in cachedStopWords)
    finder.apply_freq_filter(2)
    return finder.nbest(trigram_measures.pmi, n)

# return a formatted string of n-grams given a list of n-grams
def print_ngrams(ngram_list):
    ngrams = ''
    for ngram in ngram_list:
        ngrams += ' '.join(ngram) + ', '
    return ngrams[:-2]+ '\n\n'

In [25]:
# top 20 bigrams
bigram_collocations(tokens, 20)

[('Bahia', 'Honda'),
 ('Border', 'Patrol'),
 ('Bretton', 'Woods'),
 ('Cabot', 'Lodge'),
 ('Chiang', 'Kai-shek'),
 ('Chris', 'Getsla'),
 ('Circular', 'Note'),
 ('Clear', 'Skies'),
 ('Crimes', 'Prevention'),
 ('Cycle', 'Evaluation'),
 ('Founding', 'Fathers'),
 ('Hate', 'Crimes'),
 ('Identic', 'Circular'),
 ('Jennifer', 'Rodgers'),
 ('Jet', 'Bomber'),
 ('Kevin', 'Jett'),
 ('La', 'Abra'),
 ('Lech', 'Walesa'),
 ('Leonard', 'Wood'),
 ('Liggett', 'Meyers')]

In [26]:
# top 20 trigrams
trigram_collocations(tokens, 20)

[('Hate', 'Crimes', 'Prevention'),
 ('Identic', 'Circular', 'Note'),
 ('Fuel', 'Cycle', 'Evaluation'),
 ('Generalissimo', 'Chiang', 'Kai-shek'),
 ('Range', 'Jet', 'Bomber'),
 ('Alien', 'Property', 'Custodian'),
 ('Outer', 'Continental', 'Shelf'),
 ('Receipts', 'tures', 'icit'),
 ('Henry', 'Cabot', 'Lodge'),
 ('Bretton', 'Woods', 'Agreements'),
 ('SUPPLEMENTAL', 'LEGISLATION', 'NEEDED'),
 ('INSULAR', 'POSSESSIONS', 'Conditions'),
 ('PUBLIC', 'DOMAIN', 'EBOOKS'),
 ('CUBAN', 'PARCEL', 'POST'),
 ('Roman', 'Catholic', 'Church'),
 ('POSTAL', 'SAVINGS', 'BANKS'),
 ('Consumer', 'Price', 'Index'),
 ('Interagency', 'Coal', 'Task'),
 ('Dean', 'C.', 'Worcester'),
 ('Martin', 'Luther', 'King')]

The bigrams and trigrams showed us some new interesting information not seen from the previous sections. Unfortunately, some of the bigrams/trigrams are non-sensible, which undermine their usefulness as key phrases.  

Can we try a different approach that incorporates what we have already known from the earlier analysis?

### Attempt 1: Include Bigrams that are related to Name Entities ###

In [27]:
# interesting_entities is the list of top name entities extracted from the earlier section
interesting_entities = [ne[1] for ne in ne_output]

In [28]:
# Experiment
# take a list of top 200 bigrams, create a new bigram list 
# the new bigram list includes a bigram in the old list only if 
# at least one of its two terms is part of the top 20 name entities

bigrams = bigram_collocations(tokens, 200)
new_bigrams = []

for bigram in bigrams:
    entity_related = any(bigram[0] in entity for entity in interesting_entities) or any(
        bigram[1] in entity for entity in interesting_entities)
    if entity_related: new_bigrams += [bigram]

new_bigrams

[('La', 'Abra'), ('Schedule', 'K')]

The matching does not work very well. In addition, due to the partial matching restriction, much of the new information in the old bigram list is now lost.

### Attempt 2: Identify 'Interesting' Sentences and Parse with Chunker ###
If a sentence contains nouns that are part of the top 20 name entities, and at the same time contains verbs that have hypernyms in the top hypernyms list, that sentence might be of interest to us. I used the chunk parser to parse out the noun, proposition and verb part of the sentence.

In [29]:
# >>>>>>>>>>>>>>>>>>>> HELPER FUNCTIONS <<<<<<<<<<<<<<<<<<<<
# checks to see if a sentence contains a potentially important noun
# returns True if the tagged sentence contains a noun word 
# that is in the list of interesting name entities
# else return False
def contains_entity(tagged_sent):
    for (term, tag) in tagged_sent:
        if tag.startswith('N') and term in interesting_entities:
            return True
    return False

# checks to see if a sentence contains a potentially important verb
# returns True if the tagged sentence contains a verb word 
# that has a hypernym in the list of top verb hypernyms
# else return False
def concept_related(tagged_sent):
    for (term, tag) in tagged_sent:
        if tag.startswith('V'):
            s = wn.synsets(term.lower(), 'v')
            for syn in s:
                if any(verb_hypernym in syn.hypernyms() for verb_hypernym in verb_hypernyms_output_synset):
                    return True
    return False

In [30]:
# Experiment
# for the first 100 sentences of the text, if a sentence evaluates to true for both 
# contains_entity and concept_related,
# print out its parsed NP, PP and VP chunks

# grammar for parsing out nouns, propositions and verbs
grammar = r"""
  NP: {<DT|JJ|NN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+$} # Chunk verbs and their arguments
  """

for sent in tagged_sents[:min(100, len(tagged_sents))]:
    if concept_related(sent) and contains_entity(sent):
        print()
        print(get_chunked_text(grammar, [sent], 'NP'))
        print(get_chunked_text(grammar, [sent], 'PP'))
        print(get_chunked_text(grammar, [sent], 'VP'))

The result does not look very promising for the first 100 sentences of the test text collection. The results are redundant and noisy, and confuses more than clarifies the context. 

***

## Summary and Output ##
Overall, methods that seem to produce most meaningful information:
1. Name entities (chunking)
2. High level concepts for verbs (semantic similarity)  
<br>

After that, some other methods that give somewhat useful information:
1. High-level concepts for nouns (semantic similarity)
2. Bigrams and trigrams (collocations)
3. Interesting words by term frequencies (frequent terms) - This is not included in the final output as the list of words is mostly covered by the output of other methods   
<br>

The following attempts doesn't work with a naive approach, but might be worth exploring further:
1. Determine the usefulness of bigrams/trigrams by referencing name entities
2. Determine relevance of each sentence using name entities and high-level concepts and extract info accordingly  

### Output ###
Time to get all the pieces together in one place :)

In [31]:
# helper function to update the output with result
# ouput = output + result if the new output has total length of less than 2000 characters
def update_output(result, output, counter):
    if counter < 2000:
        result = result[:min(2000 - counter, len(result))]
        output += result
        counter = len(output)
    return output, counter

output, c = '', 0

# Name entities with their types
output, c = update_output('Key Entities of the Text Collection:\n' + ne_output_string + '\n', output, c)

# Key verb concepts for the text collection
output, c = update_output('Key Actions of the Text Collection:\n' + verb_hypernyms_output_string + '\n', output, c)

output += '\n---------- Other Words of Potential Interest ----------\n'

# Key noun concepts for the text collection
output, c = update_output('Some Concepts for Nouns:\n' + noun_hypernyms_output_string + '\n', output, c)

# Top bigrams by pmi measure
output, c = update_output('Some Bigrams:\n' + print_ngrams(bigram_collocations(tokens, 10)), output, c)

# Top trigrams by pmi measure
output, c = update_output('Some Trigrams:\n' + print_ngrams(trigram_collocations(tokens, 10)), output, c)

print(output)

Key Entities of the Text Collection:
National Congress, American Army, United States, Latin America, Federal Housing Administration, State Department, Applause, Eastern Europe, Senate, Nation, Mainland China, Treasury Department, Americans, Republic of Korea, Navy, George Washington, Arab States, Cuba, House, United Nations


Key Actions of the Text Collection:
challenge, call, express, command, mean, forbid, order, convey, protest, rede, request, provoke, out, restrict, urge, publicize, broach, guarantee, imply, elaborate


---------- Other Words of Potential Interest ----------
Some Concepts for Nouns:
fraction, termination, beginning, improvement, motion, decrease, department, increase, room, motion, production, support, position, device, integer, change, tract, blow, activity, payment

Some Bigrams:
Bahia Honda, Border Patrol, Bretton Woods, Cabot Lodge, Chiang Kai-shek, Chris Getsla, Circular Note, Clear Skies, Crimes Prevention, Cycle Evaluation

Some Trigrams:
Hate Crimes Preven